In [1]:
import os,sys
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
# os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import pandas as pd
import shutil
import pickle
import tqdm
from tqdm import tqdm
from random import shuffle
from math import log, floor
from keras.utils import *
from keras.utils.generic_utils import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *
from keras.preprocessing.image import *
from multiprocessing import *
import gensim
from gensim.models.word2vec import *
from sklearn.metrics.pairwise import *
from sklearn.model_selection import *
from sklearn.manifold import *
from sklearn.decomposition import *
from sklearn.cluster import *
import sent2vec
import re
import string
import unicodedata as udata
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter
from operator import itemgetter
from collections import OrderedDict
import random

Using TensorFlow backend.


In [2]:
samples_pid_train = pickle.load(open('./data/tree-rep-profiles-partial/TRAIN_pidNames.pkl','rb'))
samples_pid_valid = pickle.load(open('./data/tree-rep-profiles-partial/DEV_pidNames.pkl','rb'))
samples_pid_test = pickle.load(open('./data/tree-rep-profiles-partial/TEST_pidNames.pkl','rb'))

In [3]:
# all many2many pid
all_m2m_pid = []
root_dir = './data/tree-rep-profiles_one2many/'
rasmma_dir  = next(os.walk(root_dir))[1]
for dir_ in rasmma_dir:
    fam_dir = next(os.walk(root_dir + dir_))[1]
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir + dir_+ '/' + fam))[1]
        for tree in tree_dir:
            in_directory = root_dir + dir_ + '/' + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            all_m2m_pid.extend(hl_list)
all_m2m_pid = set(all_m2m_pid)

100%|██████████| 183/183 [00:00<00:00, 3362.41it/s]


In [31]:
api_li = ['LoadLibrary',
'CreateProcess',
'OpenProcess',
'ExitProcess',
'TerminateProcess',
'WinExec',
'CreateRemoteThread',
'CreateThread',
'CopyFile',
'CreateFile',
'DeleteFile',
'RegSetValue',
'RegCreateKey',
'RegDeleteKey',
'RegDeleteValue',
'RegQueryValue',
'RegEnumValue',
'WinHttpConnect',
'WinHttpOpen',
'WinHttpOpenRequest',
'WinHttpReadData',
'WinHttpSendRequest',
# 'WinHttpWriteData', #少了
'InternetOpen',
'InternetConnect',
'HttpSendRequest',
'GetUrlCacheEntryInfo']
# api_li = [x.lower() for x in api_li] #lowrer case?
len(api_li)

26

* 是否都轉小寫?
* sent2vec hyper parameters
* 

In [5]:
# profile = './data/tree-rep-profiles_one2many/small_short/6.allaple_0.8/G1478/c6d288841b9719a01bd340da55538ce43aaa74265791a41ee493cabde9d2402e_2960.profile'

# with open(profile,encoding='ISO 8859-1') as f:
#     lines = f.read()
# lines = re.sub(r'[^\x00-\x7F]+','', lines)
# lines = re.sub(r'[\x1e\x7f\x15\x10\x0c]+','', lines)
# lines.splitlines()

In [6]:
corpus = []
dil= r"[,.;\-+^()/@#?!&$:{}\\*%~\'\"]+\ *" #等號、底線被保留

root_dir = './data/tree-rep-profiles_one2many/'
rasmma_dir  = next(os.walk(root_dir))[1]
for dir_ in rasmma_dir:
    fam_dir = next(os.walk(root_dir + dir_))[1]
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir + dir_+ '/' + fam))[1]
        for tree in tree_dir:
            in_directory = root_dir + dir_ + '/' + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            for hl in hl_list:
                try:
                    if hl in samples_pid_test: #移除在test裡面的hkl
                        hl_list.remove(hl)
                    if hl in samples_pid_valid: #移除在valid裡面的hkl
                        hl_list.remove(hl)
                except ValueError:
                    pass
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
#             hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            for profile in hl_list:
                with open(profile,encoding='ISO 8859-1') as f:
                    lines = f.read()
                lines = re.sub(r'[^\x00-\x7F]+','', lines)
                lines = re.sub(r'[\x1e\x7f\x15\x10\x0c]+','', lines)
                lines = lines.splitlines()
                for line in lines:
                    temp = re.sub(dil," ",line.lower()) #lower小寫?
                    temp = temp.split(" ")
                    temp = list(filter(None, temp))
                    temp = ' '.join(temp)
                    if temp.split(' ')[0] not in api_li:
                        print('=o2m_ERR:=',profile,'=>',temp)
                        continue
                    corpus.append(temp)

root_dir = './data/tree-rep-profiles_one2one/'
rasmma_dir  = next(os.walk(root_dir))[1]
for dir_ in rasmma_dir:
    fam_dir = next(os.walk(root_dir + dir_))[1]
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir + dir_+ '/' + fam))[1]
        for tree in tree_dir:
            in_directory = root_dir + dir_ + '/' + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            for hl in hl_list:
                try:
                    if hl in all_m2m_pid: #移除已經有的pid
                        hl_list.remove(hl)
                except ValueError:
                    pass
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            for profile in hl_list:
                with open(profile,encoding='ISO 8859-1') as f:
                    lines = f.read()
                lines = re.sub(r'[^\x00-\x7F]+','', lines)
                lines = re.sub(r'[\x1e\x7f\x15\x10\x0c]+','', lines)
                lines = lines.splitlines()
                for line in lines:
                    temp = re.sub(dil," ",line.lower()) #小寫?
                    temp = temp.split(" ")
                    temp = list(filter(None, temp))
                    temp = ' '.join(temp)
                    if temp.split(' ')[0] not in api_li:
                        print('=o2o_ERR:=',profile,'=>',temp)
                        continue
                    corpus.append(temp)

with open('data/sent2vec_corpus/o2o_o2m_Sent2Vec_0523.txt','w') as f:
    f.write('\n'.join(corpus))
corpus[-10:]

100%|██████████| 166/166 [00:19<00:00,  8.67it/s]


['LoadLibrary PR SYS uxtheme DLL Ret 0',
 'LoadLibrary PR SYS oleaut32 DLL Ret 0',
 'LoadLibrary PR SYS sxs DLL Ret 0',
 'LoadLibrary PR SYS oleaut32 DLL Ret 0',
 'RegQueryValue PR HKLM soft_ms_winNT_imm PR SUBK ime file PR 0 PR 12f44c Ret 0',
 'LoadLibrary PR SYS version DLL Ret 0',
 'LoadLibrary PR SYS msctfime IME Ret 0',
 'CreateFile PR SYS IME PR GENERIC_READ PR OPEN_EXISTING PR FILE_SHARE_DELETE FILE_SHARE_READ Ret 0',
 'LoadLibrary PR SYS msctfime IME Ret 0',
 'CreateFile PR SYS IME PR GENERIC_READ PR OPEN_EXISTING PR FILE_SHARE_DELETE FILE_SHARE_READ Ret 0']

# Sent2Vec Training time 
* cd ~/Downloads/sent2vec/

## Evaluate
* TODO: 對角線相加分數要越大越好

In [32]:
with open('data/sent2vec_corpus/o2o_o2m_Sent2Vec_0523.txt','r') as f: #asp1_sentences_rep_0116
    corpus = f.read().splitlines()
corpus = sorted(corpus)
corpus[-100:]

['WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest

In [33]:
model = sent2vec.Sent2vecModel()
model.load_model('model/sent2vec_o2o_o2m_0523.bin')
emb_all = model.embed_sentences(corpus)

In [34]:
api = ""
count = 0
count_all = {}
for row in corpus:
    try:
        count_all[row.split(' ')[0]] += 1
    except KeyError:
        count_all[row.split(' ')[0]] = 1
print(len(count_all))
sorted(count_all.items())

26


[('CopyFile', 115190),
 ('CreateFile', 876116),
 ('CreateProcess', 9287),
 ('CreateRemoteThread', 23233),
 ('CreateThread', 21579),
 ('DeleteFile', 13442),
 ('ExitProcess', 9182),
 ('GetUrlCacheEntryInfo', 44),
 ('HttpSendRequest', 2120),
 ('InternetConnect', 2196),
 ('InternetOpen', 2132),
 ('LoadLibrary', 484242),
 ('OpenProcess', 32552),
 ('RegCreateKey', 202220),
 ('RegDeleteKey', 1585),
 ('RegDeleteValue', 10849),
 ('RegEnumValue', 322417),
 ('RegQueryValue', 2341392),
 ('RegSetValue', 138037),
 ('TerminateProcess', 96),
 ('WinExec', 690),
 ('WinHttpConnect', 4475),
 ('WinHttpOpen', 4487),
 ('WinHttpOpenRequest', 4474),
 ('WinHttpReadData', 4389),
 ('WinHttpSendRequest', 4488)]

In [35]:
kk = list(count_all.keys())
for k in api_li:
    if k not in kk:
        print('少了:',k)
for k in kk:
    if k not in api_li:
        print('多了',k)

In [36]:
c= 0
df = pd.DataFrame(0, index=api_li, columns=api_li)
index_ = df.index.tolist()
avg = 100
for _ in range(avg):
    c=0
    for k,v in count_all.items():
        pick = random.randrange(c,c+v) #cosine similarity的左邊
        pick = emb_all[pick,:] #sorted corpus的embedding_all
        c1 = 0
        try:
            row = index_.index(k)
        except ValueError:
            continue
        for k1,v1 in count_all.items(): # cosine similarity的右邊
            pick1 = random.randrange(c1,c1+v1)
            pick1 = emb_all[pick1,:]
            score = cosine_similarity([pick],[pick1]) #cosine_similarity
    #         print(score)
            c1 = c1+v1
            try:
                col = index_.index(k1)
                df.iloc[row,col] += float(score)
            except ValueError:
                continue
        c = c+v
df = df/avg #similarity
# df = (df-np.min(df.values))/(np.max(df.values)-np.min(df.values)) #distance normalize
df

,LoadLibrary,CreateProcess,OpenProcess,ExitProcess,TerminateProcess,WinExec,CreateRemoteThread,CreateThread,CopyFile,CreateFile,...,RegEnumValue,WinHttpConnect,WinHttpOpen,WinHttpOpenRequest,WinHttpReadData,WinHttpSendRequest,InternetOpen,InternetConnect,HttpSendRequest,GetUrlCacheEntryInfo
LoadLibrary,0.881490,0.184804,0.547542,0.343865,0.068532,0.241242,-0.013421,0.104379,0.286798,0.070740,...,-0.097004,0.093601,-0.177931,0.042693,0.025305,0.024723,0.053609,0.384317,-0.072221,-0.188735
CreateProcess,0.173830,0.841169,0.318526,0.352704,0.348104,0.831062,0.277181,0.345578,0.543799,0.497382,...,0.342824,0.202214,0.123108,0.323718,0.236384,0.086188,0.299688,0.271809,0.222320,0.208592
OpenProcess,0.534545,0.327477,0.934430,0.500439,0.290412,0.505926,0.277718,0.336593,0.321041,0.194590,...,0.012968,0.171443,-0.048250,0.206112,0.257121,0.141843,0.300226,0.472865,0.162911,-0.099228
ExitProcess,0.307096,0.363294,0.455081,0.861869,0.502732,0.427681,0.361386,0.423472,0.366269,0.282422,...,0.192368,0.266814,0.224756,0.443475,0.165412,-0.036392,0.437334,0.463161,0.074486,0.005431
TerminateProcess,0.064905,0.352722,0.308520,0.543261,0.755202,0.426964,0.573204,0.599909,0.284016,0.320159,...,0.311575,0.418551,0.170095,0.305758,0.214835,-0.085164,0.347764,0.400127,0.352787,0.292933
WinExec,0.238683,0.851376,0.512950,0.436783,0.423490,1.000000,0.237030,0.349812,0.463946,0.379874,...,0.192879,0.172305,0.023697,0.389312,0.265292,0.099141,0.308053,0.477455,0.281414,0.212757
CreateRemoteThread,-0.021700,0.274097,0.273551,0.362835,0.570626,0.241487,0.946857,0.779491,0.353213,0.547513,...,0.479941,0.517748,0.317624,0.309283,0.228316,-0.099549,0.351137,0.123057,0.269944,0.253169
CreateThread,0.091983,0.350704,0.325855,0.428382,0.605933,0.358953,0.765859,0.916770,0.416850,0.522028,...,0.446665,0.509230,0.373565,0.315727,0.292214,-0.008816,0.456481,0.260211,0.183242,0.200745
CopyFile,0.285942,0.554362,0.317016,0.373930,0.293740,0.462841,0.353165,0.417007,0.856128,0.583766,...,0.351556,0.285260,0.187743,0.238314,0.084692,0.048741,0.224947,0.129514,0.082259,0.061108
CreateFile,0.066226,0.492477,0.198764,0.280378,0.325540,0.382652,0.537177,0.541688,0.578901,0.808030,...,0.569997,0.376281,0.431112,0.246760,0.144079,0.019457,0.339221,0.009993,0.115533,0.125704


In [39]:
lib = ['LoadLibrary']
proc = ['CreateProcess', 'OpenProcess', 'ExitProcess', 'TerminateProcess', 'WinExec',
        'CreateRemoteThread', 'CreateThread']
file = ['CopyFile', 'CreateFile', 'DeleteFile']
reg = ['RegSetValue', 'RegCreateKey', 'RegDeleteKey', 'RegDeleteValue',
       'RegQueryValue', 'RegEnumValue']
net = ['WinHttpConnect', 'WinHttpOpen', 'WinHttpOpenRequest', 'WinHttpReadData', 'WinHttpSendRequest', #'WinHttpWriteData',
        'InternetOpen', 'InternetConnect', 'HttpSendRequest', 'GetUrlCacheEntryInfo']

#lower?
# lib = [x.lower() for x in lib]
# proc = [x.lower() for x in proc]
# file = [x.lower() for x in file]
# reg =[x.lower() for x in reg]
# net = [x.lower() for x in net]

# df.loc['']
x = [lib,proc,file,reg,net]
x = sum(x,[])
index_ , len(index_)

(['LoadLibrary',
  'CreateProcess',
  'OpenProcess',
  'ExitProcess',
  'TerminateProcess',
  'WinExec',
  'CreateRemoteThread',
  'CreateThread',
  'CopyFile',
  'CreateFile',
  'DeleteFile',
  'RegSetValue',
  'RegCreateKey',
  'RegDeleteKey',
  'RegDeleteValue',
  'RegQueryValue',
  'RegEnumValue',
  'WinHttpConnect',
  'WinHttpOpen',
  'WinHttpOpenRequest',
  'WinHttpReadData',
  'WinHttpSendRequest',
  'InternetOpen',
  'InternetConnect',
  'HttpSendRequest',
  'GetUrlCacheEntryInfo'],
 26)

In [40]:
up=0
down = len(index_)
down_cat = len(x)
up_cat = 0
up_cat2 = 0
lib_lib=lib_proc=lib_file=lib_reg=lib_net = 0
proc_lib=proc_proc=proc_file=proc_reg=proc_net = 0
file_lib=file_proc=file_file=file_reg=file_net = 0
reg_lib=reg_proc=reg_file=reg_reg=reg_net = 0
net_lib=net_proc=net_file=net_reg=net_net = 0
for api in index_:
    if df.loc[api].idxmax() == api:
        up+=1
    else:
        print(api)
    baseline_lib = df.loc[api,lib].mean()
    baseline_proc =  df.loc[api,proc].mean()
    baseline_file = df.loc[api,file].mean()
    baseline_reg = df.loc[api,reg].mean()
    baseline_net = df.loc[api,net].mean()
    max_value = np.max([baseline_lib,baseline_proc,baseline_file,baseline_reg,baseline_net])
    if (api in lib):
        lib_lib += baseline_lib
        lib_proc += baseline_proc
        lib_file += baseline_file
        lib_reg += baseline_reg
        lib_net += baseline_net
        if (max_value==baseline_lib):
            up_cat+=1
    elif api in proc :
        proc_lib += baseline_lib
        proc_proc += baseline_proc
        proc_file += baseline_file
        proc_reg += baseline_reg
        proc_net += baseline_net
        if max_value==baseline_proc:
            up_cat+=1
    elif api in file :
        file_lib += baseline_lib
        file_proc += baseline_proc
        file_file += baseline_file
        file_reg += baseline_reg
        file_net += baseline_net
        if max_value==baseline_file:
            up_cat+=1    
    elif api in reg:
        reg_lib += baseline_lib
        reg_proc += baseline_proc
        reg_file += baseline_file
        reg_reg += baseline_reg
        reg_net += baseline_net
        if max_value==baseline_reg:
            up_cat+=1    
    elif api in net:
        net_lib += baseline_lib
        net_proc += baseline_proc
        net_file += baseline_file
        net_reg += baseline_reg
        net_net += baseline_net
        if max_value==baseline_net:
            up_cat+=1
if np.max([lib_lib,lib_proc,lib_file,lib_reg,lib_net]) == lib_lib:
    up_cat2+=1
if np.max([proc_lib,proc_proc,proc_file,proc_reg,proc_net]) == proc_proc:
    up_cat2+=1
if np.max([file_lib,file_proc,file_file,file_reg,file_net]) == file_file:
    up_cat2+=1
if np.max([reg_lib,reg_proc,reg_file,reg_reg,reg_net]) == reg_reg:
    up_cat2+=1
if np.max([net_lib,net_proc,net_file,net_reg,net_net]) == net_net:
    up_cat2+=1
        
print('自己還原自己-ACC:',(up/down)*100,'%') #self
print('自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的):',(up_cat/down_cat)*100,'%') #micro
print('自己那類屬於自己那一類-ACC:',(up_cat2/5)*100,'%') #macro

RegQueryValue
自己還原自己-ACC: 96.15384615384616 %
自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的): 76.92307692307693 %
自己那類屬於自己那一類-ACC: 100.0 %


:~/Downloads/sent2vec$ ./fasttext sent2vec -input ~/leoqaz12_python/ANTS/seqgen_famclf/data/o2m_o2o_Sent2Vec_0521.txt -output ~/leoqaz12_python/ANTS/seqgen_famclf/model/sent2vec_o2m_o2o_0521 -minCount 0 -dim 700 -epoch 15 -lr 0.3 -wordNgrams 13 -loss ns -neg 10 -thread 35 -dropoutK 1 -minCountLabel 0 -bucket 2000000 -t 0.00001 -numCheckPoints 2
* 自己還原自己-ACC: 88.88888888888889 %
* 自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的) 81.48148148148148 %
* 自己那類屬於自己那一類-ACC 100.0 %
* 小寫的更高
***
濾掉=號、底線、轉小寫:
regqueryvalue
winhttpwritedata
* 自己還原自己-ACC: 92.5925925925926 %
* 自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的): 85.18518518518519 %
* 自己那類屬於自己那一類-ACC: 80.0 %
./fasttext sent2vec -input ~/leoqaz12_python/ANTS/seqgen_famclf/data/o2m_Sent2Vec_0521.txt -output ~/leoqaz12_python/ANTS/seqgen_famclf/model/sent2vec_o2m_0521 -minCount 0 -dim 700 -epoch 15 -lr 0.3 -wordNgrams 18 -loss ns -neg 10 -thread 35 -dropoutK 1 -minCountLabel 0 -bucket 2000000 -t 0.00001 -numCheckPoints 2
***
 ./fasttext sent2vec -input ~/leoqaz12_python/ANTS/seqgen_famclf/data/o2o_o2m_lower_woPunct_Sent2Vec_0522.txt -output ~/leoqaz12_python/ANTS/seqgen_famclf/model/sent2vec_o2o_o2m_lower_woPunct_0522 -minCount 0 -dim 700 -epoch 15 -lr 0.3 -wordNgrams 18 -loss ns -neg 10 -thread 35 -dropoutK 1 -minCountLabel 0 -bucket 2000000 -t 0.00001 -numCheckPoints 2
 * 自己還原自己-ACC: 96.15384615384616 %
 * 自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的): 88.46153846153845 %
 * 自己那類屬於自己那一類-ACC: 100.0 %
 ***
 ./fasttext sent2vec -input ~/leoqaz12_python/ANTS/seqgen_famclf/data/sent2vec_corpus/o2o_o2m_lower_woPunct_Sent2Vec_0522.txt -output ~/leoqaz12_python/ANTS/seqgen_famclf/model/sent2vec_o2o_o2m_lower_woPunct_0523 -minCount 0 -dim 700 -epoch 15 -lr 0.3 -wordNgrams 16 -loss ns -neg 10 -thread 35 -dropoutK 1 -minCountLabel 0 -bucket 2000000 -t 0.00001 -numCheckPoints 2
 * 自己還原自己-ACC: 96.15384615384616 %
* 自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的): 76.92307692307693 %
* 自己那類屬於自己那一類-ACC: 100.0 %
***
./fasttext sent2vec -input ~/leoqaz12_python/ANTS/seqgen_famclf/data/sent2vec_corpus/o2o_o2m_lower_Sent2Vec_0523.txt -output ~/leoqaz12_python/ANTS/seqgen_famclf/model/sent2vec_o2o_o2m_lower_0523 -minCount 0 -dim 700 -epoch 15 -lr 0.3 -wordNgrams 13 -loss ns -neg 10 -thread 35 -dropoutK 1 -minCountLabel 0 -bucket 2000000 -t 0.00001 -numCheckPoints 2
* 自己還原自己-ACC: 92.3076923076923 %
* 自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的): 80.76923076923077 %
* 自己那類屬於自己那一類-ACC: 100.0 %

## 